# Definição de funções e imports necessários

In [2]:
import pandas as pd
import numpy as np
import json
import excel2json
from geopy.geocoders import Nominatim
import pymongo

### Funções:

In [108]:
def readCSV (path):
    return pd.read_csv(path, sep = ";");

def readXLSX (path):
    return pd.read_excel(path, index_col=None);

def openDataBaseMongoDB ():
    client = pymongo.MongoClient("mongodb://<ADMIN>:<PASSWORDS>@criedata-shard-00-00.urzuu.mongodb.net:27017,criedata-shard-00-01.urzuu.mongodb.net:27017,criedata-shard-00-02.urzuu.mongodb.net:27017/<dbname>?ssl=true&replicaSet=atlas-6db9id-shard-0&authSource=admin&retryWrites=true&w=majority");
    db = client.schoolData;
    return db.dataINEPSchools;

def dataframeToJSONConverter (dataframe):
    dataJSON = dataframe.to_json(orient="records")
    return json.loads(dataJSON)

def deletingColumns (dataframe):
    columns = ["Região", 
               "UF", 
               "Código do Município", 
               "Nome do Município", 
               "Nome da Escola", 
               "Localização", 
               "Dependência Administrativa"];
    
    for column in columns:
        try:
            dataframe.pop(column)
        except:
            continue
    
def JSONstructure (data):    
    dataJSON = {"Taxa de não resposta": 
                {
                "Última atualização": data["Ano"],
                #"Educação Infantil": 
                    #{
                       # "Total": data["TotalI"],
                      #  "Creche": data["Creche"],
                       # "Pré-Escola": data["Pré-Escola"]
                    #},
                "Ensino Fundamental 8 e 9 anos":
                    {
                        "Total": data["TotalF"],
                        "Anos Iniciais": data["Anos Iniciais"],
                        "Anos Finais": data["Anos Finais"],
                        "1º Ano": data["1° ano"],
                        "2º Ano": data["2° ano"],
                        "3º Ano": data["3° ano"],
                        "4º Ano": data["4° ano"],
                        "5º Ano": data["5° ano"],
                        "6º Ano": data["6° ano"],
                        "7º Ano": data["7° ano"],
                        "8º Ano": data["8° ano"],
                        "9º Ano": data["9° ano"]
                        #"Turmas Multietapa, Multi ou Correção de Fluxo 2": data["Turmas Multietapa, Multi ou Correção de Fluxo 2"] 
                    },
                "Ensino Médio": 
                    {
                        "Total": data["TotalM"],
                        "1ª série": data["1ª série"],
                        "2ª série": data["2ª série"],
                        "3ª série": data["3ª série"],
                        "4ª série": data["4ª série"],
                        "Não-Seriado": data["Não-Seriado"]
                    }
            }};

    return dataJSON;

def filterPelotas (dataframe):
    filter = dataframe["Nome do Município"] == "Pelotas"
    
    filterPelotas = dataframe[filter]
    filterPelotas.reset_index(drop=True, inplace=True)
    
    return filterPelotas

def insertIntoDB (dataframe, collection):
    for index, row in dataframe.iterrows():
        teste1 = JSONstructure(row);

        collection.update_one({"Código INEP": row["Código da Escola"]}, {"$set": teste1});

### Conexão com o BD:

In [5]:
collection = openDataBaseMongoDB()

# ETL: Informações das Escolas

In [23]:
infoSchools = readCSV(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\InformacoesGeraisDasEscolas.csv');
infoSchools.head()

,Restrição de Atendimento,Escola,Código INEP,UF,Município,Localização,Localidade Diferenciada,Categoria Administrativa,Endereço,Telefone,Dependência Administrativa,Categoria Escola Privada,Conveniada Poder Público,Regulamentação pelo Conselho de Educação,Porte da Escola,Etapas e Modalidade de Ensino Oferecidas,Outras Ofertas Educacionais,Latitude,Longitude
0,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,COLEGIO TIRADENTES PELOTAS,43000304,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Pública,"AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003...",(53) 32812866,Estadual,Não Informado,Não,Sim,Entre 201 e 500 matrículas de escolarização,Ensino Médio,NaN,-31.75701,-52.37308
1,ESCOLA PARALISADA,EEI NOSSOS AMIGUINHOS,43001084,RS,Pelotas,Urbana,Não Informado,Privada,"RUA VOLUNTARIOS DA PATRIA, 1927. CENTRO. 96015...",(53) 30256230,Privada,Não Informado,Não,NaN,NaN,NaN,NaN,NaN,NaN
2,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI CRIANCA AMADA,43001165,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"AVENIDA DOMINGOS JOSE DE ALMEIDA, 1077 AREAL. ...",(53) 30282009,Privada,Particular,Não,Sim,Entre 51 e 200 matrículas de escolarização,Educação Infantil,NaN,NaN,NaN
3,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI MUNDO DO MICKEY,43001173,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"RUA BALDOMERO TRAPAGA, 132 PORTO. 96075-540 Pe...",(53) 32279908,Privada,Particular,Não,Sim,Até 50 matrículas de escolarização,Educação Infantil,NaN,-31.77139,-52.32994
4,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI CHAPEUZINHO VERMELHO,43002145,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"RUA BARAO DE SANTA TECLA, 621 CENTRO. 96010-14...",(53) 81265782,Privada,Particular,Não,Sim,Até 50 matrículas de escolarização,Educação Infantil,NaN,-31.76221,-52.34358


In [24]:
infoSchools.shape

(302, 19)

In [25]:
infoSchools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 19 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Restrição de Atendimento                  302 non-null    object 
 1   Escola                                    302 non-null    object 
 2   Código INEP                               302 non-null    int64  
 3   UF                                        302 non-null    object 
 4   Município                                 302 non-null    object 
 5   Localização                               302 non-null    object 
 6   Localidade Diferenciada                   302 non-null    object 
 7   Categoria Administrativa                  302 non-null    object 
 8   Endereço                                  302 non-null    object 
 9   Telefone                                  292 non-null    object 
 10  Dependência Administrativa            

In [26]:
infoSchools.isnull().sum()

Restrição de Atendimento                      0
Escola                                        0
Código INEP                                   0
UF                                            0
Município                                     0
Localização                                   0
Localidade Diferenciada                       0
Categoria Administrativa                      0
Endereço                                      0
Telefone                                     10
Dependência Administrativa                    0
Categoria Escola Privada                      0
Conveniada Poder Público                      0
Regulamentação pelo Conselho de Educação     63
Porte da Escola                              63
Etapas e Modalidade de Ensino Oferecidas     66
Outras Ofertas Educacionais                 212
Latitude                                    104
Longitude                                   104
dtype: int64

### Processo de Geocoding:

In [27]:
infoSchools["Endereço"][2]

'AVENIDA DOMINGOS JOSE DE ALMEIDA, 1077 AREAL. 96085-470 Pelotas - RS.'

In [82]:
geolocator = Nominatim(user_agent = "ScoolarData")
latitude = []
longitude = []

for local in collectionINEP["Endereço"]:
    try:
        location = geolocator.geocode(local)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        continue

### Conversão do *dataframe* para json:

In [28]:
schoolData = dataframeToJSONConverter (infoSchools)
schoolData[0]

{'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308}

In [59]:
collection.insert_many(schoolData)

# ETL: Média de estudates por escola

In [51]:
studentsAVG = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\MediaDeAlunosPorTurma.xlsx')

In [52]:
studentsAVG.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,--,...,--,--,--,7,--,--,--,--,--,--
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,--,...,--,--,--,--,--,--,--,--,--,--
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,--,...,--,--,--,10,--,--,--,--,--,--
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,10,...,7,15,8,23.5,--,--,--,--,--,--
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,16,...,20,26,18,--,--,--,--,--,--,--


In [53]:
studentsAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176386 entries, 0 to 176385
Data columns (total 31 columns):
 #   Column                                           Non-Null Count   Dtype 
---  ------                                           --------------   ----- 
 0   Ano                                              176386 non-null  int64 
 1   Região                                           176386 non-null  object
 2   UF                                               176386 non-null  object
 3   Código do Município                              176386 non-null  int64 
 4   Nome do Município                                176386 non-null  object
 5   Código da Escola                                 176386 non-null  int64 
 6   Nome da Escola                                   176386 non-null  object
 7   Localização                                      176386 non-null  object
 8   Dependência Administrativa                       176386 non-null  object
 9   TotalI                    

In [54]:
studentsAVG_Pelotas = filterPelotas(studentsAVG)
studentsAVG_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,--,...,--,--,--,--,26.6,25,26,30,--,--
1,2019,Sul,RS,4314407,Pelotas,43001165,EEI CRIANCA AMADA,Urbana,Privada,14,...,--,--,--,--,--,--,--,--,--,--
2,2019,Sul,RS,4314407,Pelotas,43001173,EEI MUNDO DO MICKEY,Urbana,Privada,3,...,--,--,--,--,--,--,--,--,--,--
3,2019,Sul,RS,4314407,Pelotas,43002145,EEI CHAPEUZINHO VERMELHO,Urbana,Privada,3.5,...,--,--,--,--,--,--,--,--,--,--
4,2019,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,--,...,30.5,37,--,--,--,--,--,--,--,--


In [55]:
studentsAVG_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 31 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Ano                                              228 non-null    int64 
 1   Região                                           228 non-null    object
 2   UF                                               228 non-null    object
 3   Código do Município                              228 non-null    int64 
 4   Nome do Município                                228 non-null    object
 5   Código da Escola                                 228 non-null    int64 
 6   Nome da Escola                                   228 non-null    object
 7   Localização                                      228 non-null    object
 8   Dependência Administrativa                       228 non-null    object
 9   TotalI                                     

In [56]:
deletingColumns(studentsAVG_Pelotas)

In [57]:
studentsAVG_Pelotas.head()

,Ano,Código da Escola,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,43000304,--,--,--,--,--,--,--,--,...,--,--,--,--,26.6,25,26,30,--,--
1,2019,43001165,14,13.6,15,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
2,2019,43001173,3,6,1.5,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
3,2019,43002145,3.5,4.3,2.7,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
4,2019,43003222,--,--,--,29.4,27.3,34.7,26.7,27.7,...,30.5,37,--,--,--,--,--,--,--,--


### Inserindo no BD:

In [60]:
insertIntoDB(studentsAVG_Pelotas, collection)

In [61]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2445bd647ad0c134f5893e'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Media de alunos por turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'

# ETL: Média horas-aula diária por escola

In [68]:
hoursAVG = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\MediaHorasAulaDiaria.xlsx')

In [69]:
hoursAVG.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,4.0,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,4.0,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,NaN,...,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024828,EMEIEF IZIDORO STEDILE,Urbana,Municipal,4.0,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024917,EMEIEF JOSE BASILIO DA GAMA,Rural,Municipal,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
hoursAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152509 entries, 0 to 152508
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Ano                         152509 non-null  int64  
 1   Região                      152509 non-null  object 
 2   UF                          152509 non-null  object 
 3   Código do Município         152509 non-null  int64  
 4   Nome do Município           152509 non-null  object 
 5   Código da Escola            152509 non-null  int64  
 6   Nome da Escola              152509 non-null  object 
 7   Localização                 152509 non-null  object 
 8   Dependência Administrativa  152509 non-null  object 
 9   TotalI                      94830 non-null   float64
 10  Creche                      59884 non-null   float64
 11  Pré-Escola                  82102 non-null   float64
 12  TotalF                      98340 non-null   float64
 13  Anos Iniciais 

In [71]:
hoursAVG_Pelotas = filterPelotas(hoursAVG)
hoursAVG_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,NaN,...,NaN,NaN,NaN,NaN,8.9,8.9,8.9,8.9,NaN,NaN
1,2019,Sul,RS,4314407,Pelotas,43001165,EEI CRIANCA AMADA,Urbana,Privada,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Sul,RS,4314407,Pelotas,43001173,EEI MUNDO DO MICKEY,Urbana,Privada,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Sul,RS,4314407,Pelotas,43002145,EEI CHAPEUZINHO VERMELHO,Urbana,Privada,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,NaN,...,4.1,4.1,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
hoursAVG_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ano               224 non-null    int64  
 1   Código da Escola  224 non-null    int64  
 2   TotalI            161 non-null    float64
 3   Creche            95 non-null     float64
 4   Pré-Escola        157 non-null    float64
 5   TotalF            119 non-null    float64
 6   Anos Iniciais     112 non-null    float64
 7   Anos Finais       93 non-null     float64
 8   1° ano            107 non-null    float64
 9   2° ano            105 non-null    float64
 10  3° ano            108 non-null    float64
 11  4° ano            106 non-null    float64
 12  5° ano            109 non-null    float64
 13  6° ano            89 non-null     float64
 14  7° ano            88 non-null     float64
 15  8° ano            86 non-null     float64
 16  9° ano            87 non-null     float64
 1

In [72]:
deletingColumns(hoursAVG_Pelotas)

In [73]:
hoursAVG_Pelotas.head()

,Ano,Código da Escola,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,43000304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.9,8.9,8.9,8.9,NaN,NaN
1,2019,43001165,12.0,12.0,12.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,43001173,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,43002145,5.5,5.4,5.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,43003222,NaN,NaN,NaN,4.2,4.3,4.1,4.3,4.3,...,4.1,4.1,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Inserindo no BD:

In [74]:
insertIntoDB(hoursAVG_Pelotas, collection)

In [75]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2445bd647ad0c134f5893e'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Media de alunos por turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'

# ETL: Taxa de Distorção de Idade-Série

In [6]:
distortionRate = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\TaxaDistorcaoIdadeSerie.xlsx')

In [7]:
distortionRate.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalF,...,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,14.3,...,0,--,--,--,--,--,--,--,--,--
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,5,...,0,--,--,--,--,--,--,--,--,--
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,13.7,...,18.2,0,28.6,46.7,25,--,--,--,--,--
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,13.8,...,9.1,21.2,20,19.2,38.9,--,--,--,--,--
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,15.6,...,14.1,19,22.3,14.3,--,--,--,--,--,--


In [8]:
distortionRate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132428 entries, 0 to 132427
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Ano                         132428 non-null  int64 
 1   Região                      132428 non-null  object
 2   UF                          132428 non-null  object
 3   Código do Município         132428 non-null  int64 
 4   Nome do Município           132428 non-null  object
 5   Código da Escola            132428 non-null  int64 
 6   Nome da Escola              132428 non-null  object
 7   Localização                 132428 non-null  object
 8   Dependência Administrativa  132428 non-null  object
 9   TotalF                      132428 non-null  object
 10  Anos Iniciais               132428 non-null  object
 11  Anos Finais                 132428 non-null  object
 12  1° ano                      132428 non-null  object
 13  2° ano                      1

In [9]:
distortionRate_Pelotas = filterPelotas(distortionRate)
distortionRate_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalF,...,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,--,...,--,--,--,--,--,13.1,6.7,23.1,8.3,--
1,2019,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,1.3,...,1.3,2.7,1.6,4.1,--,--,--,--,--,--
2,2019,Sul,RS,4314407,Pelotas,43012396,ESC EST ENS MED COLONIA DE PESCADORES Z 3,Rural,Estadual,--,...,--,--,--,--,--,79.5,93.1,50,55.6,--
3,2019,Sul,RS,4314407,Pelotas,43065376,EMEF WALDEMAR DENZER,Rural,Municipal,18,...,14.3,31.3,38.9,25,18.8,--,--,--,--,--
4,2019,Sul,RS,4314407,Pelotas,43100007,COLEGIO GONZAGA,Urbana,Privada,2,...,0.9,0,1.3,5.6,4.4,6.5,10,7.4,1.7,--


In [10]:
distortionRate_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Ano                         135 non-null    int64 
 1   Região                      135 non-null    object
 2   UF                          135 non-null    object
 3   Código do Município         135 non-null    int64 
 4   Nome do Município           135 non-null    object
 5   Código da Escola            135 non-null    int64 
 6   Nome da Escola              135 non-null    object
 7   Localização                 135 non-null    object
 8   Dependência Administrativa  135 non-null    object
 9   TotalF                      135 non-null    object
 10  Anos Iniciais               135 non-null    object
 11  Anos Finais                 135 non-null    object
 12  1° ano                      135 non-null    object
 13  2° ano                      135 non-null    object

In [12]:
deletingColumns(distortionRate_Pelotas)
distortionRate_Pelotas.head()

,Ano,Código da Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,3° ano,4° ano,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,43000304,--,--,--,--,--,--,--,--,--,--,--,--,13.1,6.7,23.1,8.3,--
1,2019,43003222,1.3,0.5,2.9,0,0,1.1,0,1.3,2.7,1.6,4.1,--,--,--,--,--,--
2,2019,43012396,--,--,--,--,--,--,--,--,--,--,--,--,79.5,93.1,50,55.6,--
3,2019,43065376,18,3.1,29.1,0,0,0,0,14.3,31.3,38.9,25,18.8,--,--,--,--,--
4,2019,43100007,2,1.2,3,0,3.5,0.8,1.1,0.9,0,1.3,5.6,4.4,6.5,10,7.4,1.7,--


### Teste JSON:

In [14]:
rowData = distortionRate_Pelotas.iloc[ 0 , : ]
rowData

Ano                     2019
Código da Escola    43000304
TotalF                    --
Anos Iniciais             --
Anos Finais               --
1° ano                    --
2° ano                    --
3° ano                    --
4° ano                    --
5° ano                    --
6° ano                    --
7° ano                    --
8° ano                    --
9° ano                    --
TotalM                  13.1
1ª série                 6.7
2ª série                23.1
3ª série                 8.3
4ª série                  --
Name: 0, dtype: object

In [17]:
JSONstructure(rowData)

{'Média Horas-Aula diárias': {'Última atualização': 2019,
  'Ensino Fundamental 8 e 9 anos': {'Total': '--',
   'Anos Iniciais': '--',
   'Anos Finais': '--',
   '1º Ano': '--',
   '2º Ano': '--',
   '3º Ano': '--',
   '4º Ano': '--',
   '5º Ano': '--',
   '6º Ano': '--',
   '7º Ano': '--',
   '8º Ano': '--',
   '9º Ano': '--'},
  'Ensino Médio': {'Total': 13.1,
   '1ª série': 6.7,
   '2ª série': 23.1,
   '3ª série': 8.3,
   '4ª série': '--'}}}

## Inserindo no BD:

In [78]:
insertIntoDB(distortionRate_Pelotas, collection)

In [79]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2445bd647ad0c134f5893e'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Media de alunos por turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'

# ETL: Taxa de Rendimento

In [82]:
yieldRate = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\TaxaRendimento.xlsx')

In [83]:
yieldRate.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalFA,...,6° anoO,7° anoO,8° anoO,9° anoO,TotalMO,1ª sérieO,2ª sérieO,3ª sérieO,4ª sérieO,Não-SeriadoO
0,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,90.9,...,--,--,--,--,--,--,--,--,--,--
1,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,92.1,...,--,--,--,--,--,--,--,--,--,--
2,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,92.5,...,14.3,13.3,16.7,8.3,--,--,--,--,--,--
3,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,94.4,...,4,0,0,35.3,--,--,--,--,--,--
4,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,85,...,0,0,0,--,--,--,--,--,--,--


In [84]:
yieldRate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134457 entries, 0 to 134456
Data columns (total 63 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Ano                         134457 non-null  int64 
 1   Região                      134457 non-null  object
 2   UF                          134457 non-null  object
 3   Código do Município         134457 non-null  int64 
 4   Nome do Município           134457 non-null  object
 5   Código da Escola            134457 non-null  int64 
 6   Nome da Escola              134457 non-null  object
 7   Localização                 134457 non-null  object
 8   Dependência Administrativa  134457 non-null  object
 9   TotalFA                     134457 non-null  object
 10  Anos IniciaisA              134457 non-null  object
 11  Anos FinaisA                134457 non-null  object
 12  1° anoA                     134457 non-null  object
 13  2° anoA                     1

In [85]:
yieldRate_Pelotas = filterPelotas(yieldRate)
yieldRate_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalFA,...,6° anoO,7° anoO,8° anoO,9° anoO,TotalMO,1ª sérieO,2ª sérieO,3ª sérieO,4ª sérieO,Não-SeriadoO
0,2018,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,--,...,--,--,--,--,0,0,0,0,--,--
1,2018,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,99,...,0,0,0,--,--,--,--,--,--,--
2,2018,Sul,RS,4314407,Pelotas,43012396,ESC EST ENS MED COLONIA DE PESCADORES Z 3,Rural,Estadual,--,...,--,--,--,--,50,51.5,42.9,53.8,--,--
3,2018,Sul,RS,4314407,Pelotas,43065376,EMEF WALDEMAR DENZER,Rural,Municipal,88.3,...,4.5,0,4.6,10,--,--,--,--,--,--
4,2018,Sul,RS,4314407,Pelotas,43100007,COLEGIO GONZAGA,Urbana,Privada,97.3,...,0,0,0,0,0,0,0,0,--,--


In [98]:
yieldRate_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 56 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Ano               136 non-null    int64 
 1   Código da Escola  136 non-null    int64 
 2   TotalFA           136 non-null    object
 3   Anos IniciaisA    136 non-null    object
 4   Anos FinaisA      136 non-null    object
 5   1° anoA           136 non-null    object
 6   2° anoA           136 non-null    object
 7   3° anoA           136 non-null    object
 8   4° anoA           136 non-null    object
 9   5° anoA           136 non-null    object
 10  6° anoA           136 non-null    object
 11  7° anoA           136 non-null    object
 12  8° anoA           136 non-null    object
 13  9° anoA           136 non-null    object
 14  TotalMA           136 non-null    object
 15  1ª sérieA         136 non-null    object
 16  2ª sérieA         136 non-null    object
 17  3ª sérieA       

In [86]:
deletingColumns(yieldRate_Pelotas)

In [87]:
yieldRate_Pelotas.head()

,Ano,Código da Escola,TotalFA,Anos IniciaisA,Anos FinaisA,1° anoA,2° anoA,3° anoA,4° anoA,5° anoA,...,6° anoO,7° anoO,8° anoO,9° anoO,TotalMO,1ª sérieO,2ª sérieO,3ª sérieO,4ª sérieO,Não-SeriadoO
0,2018,43000304,--,--,--,--,--,--,--,--,...,--,--,--,--,0,0,0,0,--,--
1,2018,43003222,99,100,97.1,100,100,100,100,100,...,0,0,0,--,--,--,--,--,--,--
2,2018,43012396,--,--,--,--,--,--,--,--,...,--,--,--,--,50,51.5,42.9,53.8,--,--
3,2018,43065376,88.3,96.2,80.3,100,100,100,92.3,92,...,4.5,0,4.6,10,--,--,--,--,--,--
4,2018,43100007,97.3,99.8,93.8,100,100,100,99,100,...,0,0,0,0,0,0,0,0,--,--


In [88]:
yieldRate_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 56 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Ano               136 non-null    int64 
 1   Código da Escola  136 non-null    int64 
 2   TotalFA           136 non-null    object
 3   Anos IniciaisA    136 non-null    object
 4   Anos FinaisA      136 non-null    object
 5   1° anoA           136 non-null    object
 6   2° anoA           136 non-null    object
 7   3° anoA           136 non-null    object
 8   4° anoA           136 non-null    object
 9   5° anoA           136 non-null    object
 10  6° anoA           136 non-null    object
 11  7° anoA           136 non-null    object
 12  8° anoA           136 non-null    object
 13  9° anoA           136 non-null    object
 14  TotalMA           136 non-null    object
 15  1ª sérieA         136 non-null    object
 16  2ª sérieA         136 non-null    object
 17  3ª sérieA       

### Teste JSON:

In [90]:
rowData = yieldRate_Pelotas.iloc[ 0 , : ]
rowData

Ano                     2018
Código da Escola    43000304
TotalFA                   --
Anos IniciaisA            --
Anos FinaisA              --
1° anoA                   --
2° anoA                   --
3° anoA                   --
4° anoA                   --
5° anoA                   --
6° anoA                   --
7° anoA                   --
8° anoA                   --
9° anoA                   --
TotalMA                 92.6
1ª sérieA                 96
2ª sérieA               84.7
3ª sérieA                100
4ª sérieA                 --
Não-SeriadoA              --
TotalFR                   --
Anos IniciaisR            --
Anos FinaisR              --
1° anoR                   --
2° anoR                   --
3° anoR                   --
4° anoR                   --
5° anoR                   --
6° anoR                   --
7° anoR                   --
8° anoR                   --
9° anoR                   --
TotalMR                  7.4
1ª sérieR                  4
2ª sérieR     

In [93]:
JSONstructure(rowData)

{'Taxa de rendimento escolar': {'Última atualização': 2018,
  'Ensino Fundamental 8 e 9 anos': {'Taxa de aprovação': {'Total': '--',
    'Anos Iniciais': '--',
    'Anos Finais': '--',
    '1º Ano': '--',
    '2º Ano': '--',
    '3º Ano': '--',
    '4º Ano': '--',
    '5º Ano': '--',
    '6º Ano': '--',
    '7º Ano': '--',
    '8º Ano': '--',
    '9º Ano': '--'},
   'Taxa de reprovação': {'Total': '--',
    'Anos Iniciais': '--',
    'Anos Finais': '--',
    '1º Ano': '--',
    '2º Ano': '--',
    '3º Ano': '--',
    '4º Ano': '--',
    '5º Ano': '--',
    '6º Ano': '--',
    '7º Ano': '--',
    '8º Ano': '--',
    '9º Ano': '--'},
   'Taxa de abandono': {'Total': '--',
    'Anos Iniciais': '--',
    'Anos Finais': '--',
    '1º Ano': '--',
    '2º Ano': '--',
    '3º Ano': '--',
    '4º Ano': '--',
    '5º Ano': '--',
    '6º Ano': '--',
    '7º Ano': '--',
    '8º Ano': '--',
    '9º Ano': '--'}},
  'Ensino Médio': {'Taxa de aprovação': {'Total': 92.6,
    '1ª série': 96,
    '2ª sér

### Inserindo no BD:

In [95]:
insertIntoDB(yieldRate_Pelotas, collection)

In [96]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2445bd647ad0c134f5893e'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Media de alunos por turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'

# ETL: Taxa de não resposta

In [99]:
noReply = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\TaxaDeNaoResposta.xlsx')

In [100]:
noReply.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalF,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,0,...,--,--,--,--,--,--,--,--,--,--
1,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,0,...,--,--,--,--,--,--,--,--,--,--
2,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,0,...,0,0,0,0,--,--,--,--,--,--
3,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,1,...,0,0,5.6,5.6,--,--,--,--,--,--
4,2018,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,1.5,...,1.5,4.5,2.3,--,--,--,--,--,--,--


In [101]:
noReply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134457 entries, 0 to 134456
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Ano                         134457 non-null  int64 
 1   Região                      134457 non-null  object
 2   UF                          134457 non-null  object
 3   Código do Município         134457 non-null  int64 
 4   Nome do Município           134457 non-null  object
 5   Código da Escola            134457 non-null  int64 
 6   Nome da Escola              134457 non-null  object
 7   Localização                 134457 non-null  object
 8   Dependência Administrativa  134457 non-null  object
 9   TotalF                      134457 non-null  object
 10  Anos Iniciais               134457 non-null  object
 11  Anos Finais                 134457 non-null  object
 12  1° ano                      134457 non-null  object
 13  2° ano                      1

In [102]:
noReply_Pelotas = filterPelotas(noReply)
noReply_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalF,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2018,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,--,...,--,--,--,--,0,0,0,0,--,--
1,2018,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,0,...,0,0,0,--,--,--,--,--,--,--
2,2018,Sul,RS,4314407,Pelotas,43012396,ESC EST ENS MED COLONIA DE PESCADORES Z 3,Rural,Estadual,--,...,--,--,--,--,0,0,0,0,--,--
3,2018,Sul,RS,4314407,Pelotas,43065376,EMEF WALDEMAR DENZER,Rural,Municipal,0,...,0,0,0,0,--,--,--,--,--,--
4,2018,Sul,RS,4314407,Pelotas,43100007,COLEGIO GONZAGA,Urbana,Privada,0.6,...,1.3,0,0.9,0,5,5.6,6.3,0,--,--


In [105]:
noReply_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Ano                         136 non-null    int64 
 1   Região                      136 non-null    object
 2   UF                          136 non-null    object
 3   Código do Município         136 non-null    int64 
 4   Nome do Município           136 non-null    object
 5   Código da Escola            136 non-null    int64 
 6   Nome da Escola              136 non-null    object
 7   Localização                 136 non-null    object
 8   Dependência Administrativa  136 non-null    object
 9   TotalF                      136 non-null    object
 10  Anos Iniciais               136 non-null    object
 11  Anos Finais                 136 non-null    object
 12  1° ano                      136 non-null    object
 13  2° ano                      136 non-null    object

In [106]:
deletingColumns(noReply_Pelotas)

In [107]:
noReply_Pelotas.head()

,Ano,Código da Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,3° ano,4° ano,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2018,43000304,--,--,--,--,--,--,--,--,--,--,--,--,0,0,0,0,--,--
1,2018,43003222,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,--
2,2018,43012396,--,--,--,--,--,--,--,--,--,--,--,--,0,0,0,0,--,--
3,2018,43065376,0,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--
4,2018,43100007,0.6,0.7,0.5,0.8,0.7,2.4,0,0,1.3,0,0.9,0,5,5.6,6.3,0,--,--


### Teste JSON:

In [109]:
rowData = noReply_Pelotas.iloc[ 0 , : ]
rowData

Ano                     2018
Código da Escola    43000304
TotalF                    --
Anos Iniciais             --
Anos Finais               --
1° ano                    --
2° ano                    --
3° ano                    --
4° ano                    --
5° ano                    --
6° ano                    --
7° ano                    --
8° ano                    --
9° ano                    --
TotalM                     0
1ª série                   0
2ª série                   0
3ª série                   0
4ª série                  --
Não-Seriado               --
Name: 0, dtype: object

In [110]:
JSONstructure(rowData)

{'Taxa de rendimento escolar': {'Última atualização': 2018,
  'Ensino Fundamental 8 e 9 anos': {'Total': '--',
   'Anos Iniciais': '--',
   'Anos Finais': '--',
   '1º Ano': '--',
   '2º Ano': '--',
   '3º Ano': '--',
   '4º Ano': '--',
   '5º Ano': '--',
   '6º Ano': '--',
   '7º Ano': '--',
   '8º Ano': '--',
   '9º Ano': '--'},
  'Ensino Médio': {'Total': 0,
   '1ª série': 0,
   '2ª série': 0,
   '3ª série': 0,
   '4ª série': '--',
   'Não-Seriado': '--'}}}

### Inserindo no BD:

In [112]:
insertIntoDB(noReply_Pelotas, collection)

In [113]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2445bd647ad0c134f5893e'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Media de alunos por turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'